In [24]:
from state import AgentState

from agents.legal_risk_analysis_agent import legal_risk_analysis_agent
import agents.scored_agent
from agents.startup_explorer_agent import startup_explorer_agent
from agents.tech_analysis_agent import tech_analysis_agent

In [49]:
import asyncio
import random

async def tech_analysis_agent(company):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Tech] {company}의 기술 분석 결과입니다."

async def founder_explorer_agent(company, domain, country):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Founder] {company} 창업자 평판 분석 완료."

async def market_analysis_agent(domain, country):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Market] {domain}의 시장 분석 결과입니다."

async def competitor_analysis_agent(company, domain, country):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Competitor] {company} 경쟁사 분석 결과입니다."

async def legal_risk_analysis_agent(company, domain, country, tech_result):
    await asyncio.sleep(random.uniform(0.5, 1.5))
    return f"[Competitor] {company} 경쟁사 분석 결과입니다."

In [50]:
import asyncio

async def analyze_company(company, domain, country):
    # tech_analysis 백그라운드 실행
    tech_task = asyncio.create_task(tech_analysis_agent(company))

    # 나머지 에이전트 동시 실행
    founder_task = asyncio.create_task(founder_explorer_agent(company, domain, country))
    competitor_task = asyncio.create_task(competitor_analysis_agent(company, domain, country))

    # tech 완료 기다렸다가 legal 시작
    tech_result = await tech_task
    legal_result = await legal_risk_analysis_agent(company, domain, country, tech_result)

    # 나머지 결과 대기
    founder_result = await founder_task
    competitor_result = await competitor_task

    return {
        "company" : company, 
        "tech_summary": tech_result,
        "founder_reputation": founder_result,
        # "market_analysis": market_result,
        "legal_risk": legal_result,
        "competitor_info": competitor_result
    }

In [51]:
result = await analyze_company("뤼튼테크놀로지", "ai chatbot", "global")
result

{'company': '뤼튼테크놀로지',
 'tech_summary': '[Tech] 뤼튼테크놀로지의 기술 분석 결과입니다.',
 'founder_reputation': '[Founder] 뤼튼테크놀로지 창업자 평판 분석 완료.',
 'legal_risk': '[Competitor] 뤼튼테크놀로지 경쟁사 분석 결과입니다.',
 'competitor_info': '[Competitor] 뤼튼테크놀로지 경쟁사 분석 결과입니다.'}

In [52]:
state: AgentState = {
    "domain": "AI Chatbot",
    "country": "Global",
    "startup_list": ["뤼튼테크놀로지", "Qure.ai"],
    "startup_profiles": {},
    "tech_summary": {},
    "founder_reputation": {},
    "market_analysis": {},
    "legal_risk": {},
    "competitor_info": {},
    "investment_decision": {},
    "final_report": {},
}

In [53]:
async def analyze_all_companies(state):
    companies = state["startup_list"]
    domain = state["domain"]
    country = state["country"]

    tech_summary = {}
    founder_reputation = {}
    legal_risk = {}
    competitor_info = {}

    # 1. 각 기업별 분석 비동기 태스크 준비
    company_tasks = [
        analyze_company(company, domain, country)
        for company in companies
    ]

    # 2. market_analysis는 단 한 번만 실행
    market_analysis_task = asyncio.create_task(market_analysis_agent(domain, country))

    # 3. 모든 작업 병렬 실행
    company_results = await asyncio.gather(*company_tasks)
    market_result = await market_analysis_task

    # 4. 기업별 결과 정리
    for i, company in enumerate(companies):
        tech_summary[company] = company_results[i]["tech_summary"]
        founder_reputation[company] = company_results[i]["founder_reputation"]
        legal_risk[company] = company_results[i]["legal_risk"]
        competitor_info[company] = company_results[i]["competitor_info"]

    # 5. market_analysis는 하나의 문자열만 반환
    return {
        "tech_summary": tech_summary,
        "founder_reputation": founder_reputation,
        "market_analysis": market_result, 
        "legal_risk": legal_risk,
        "competitor_info": competitor_info,
    }

In [54]:
result = await analyze_all_companies(state)
result

{'tech_summary': {'뤼튼테크놀로지': '[Tech] 뤼튼테크놀로지의 기술 분석 결과입니다.',
  'Qure.ai': '[Tech] Qure.ai의 기술 분석 결과입니다.'},
 'founder_reputation': {'뤼튼테크놀로지': '[Founder] 뤼튼테크놀로지 창업자 평판 분석 완료.',
  'Qure.ai': '[Founder] Qure.ai 창업자 평판 분석 완료.'},
 'market_analysis': '[Market] AI Chatbot의 시장 분석 결과입니다.',
 'legal_risk': {'뤼튼테크놀로지': '[Competitor] 뤼튼테크놀로지 경쟁사 분석 결과입니다.',
  'Qure.ai': '[Competitor] Qure.ai 경쟁사 분석 결과입니다.'},
 'competitor_info': {'뤼튼테크놀로지': '[Competitor] 뤼튼테크놀로지 경쟁사 분석 결과입니다.',
  'Qure.ai': '[Competitor] Qure.ai 경쟁사 분석 결과입니다.'}}